### Create region masks for different ocean basins

In [ ]:
# Data analysis libraries 
import sys
import xarray as xr
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.path
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter


In [ ]:
def get_mask(poly_verts, lsmask):
    '''create a mask from polygon vertices.
    
    poly_verts: polygon vertices, List [(lon,lat),...]
    
    lsmask: land mask, DataArray [nlon x nlat]
    
    '''
    x, y = np.meshgrid(lsmask.lon.values, lsmask.lat.values)
    x, y = x.flatten(), y.flatten()
    points = np.vstack((x,y)).T
    path = matplotlib.path.Path(poly_verts)
    grid = path.contains_points(points)
    grid = grid.reshape((lsmask.lat.shape[0],lsmask.lon.shape[0]))

    mask = xr.DataArray(data=grid, dims=['lat', 'lon'],
                                 coords=dict(
                                     lat=('lat', lsmask.lat.values),
                                     lon=('lon', lsmask.lon.values))
                                )
    mask = mask.where(mask==1, other=np.nan)
    return mask


### Load landmask

In [ ]:
path = 'https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.oisst.v2.highres/lsmask.oisst.nc'
mask = xr.open_dataset(path, use_cftime=True) 

# make longitude cyclic
east = np.arange(.125, 180.125, .25)
west = np.arange(-180.125, -.125, .25)
new_lon = np.concatenate((west, east))
mask['lon'] = new_lon

lsmask = mask.lsmask.isel(time=0).where((mask.lat<65) & (mask.lat>-70), drop=False, other=0) 
lsmask.plot()

### North Pacific

In [ ]:
offset = 360
poly_verts_east = [(142,-20), 
                   (142,-5), 
                   (130,0), 
                   (125,5), 
                   (125,10),
                   (120,20), 
                   (115,40),
                   (120,66),
                   (180,66),
                   (180,-20),
                  ]

poly_verts_west = [(240-offset,66),
                   (240-offset,40),
                   (250-offset,30),
                   (260-offset,20),
                   (270-offset,15),
                   (276-offset,12),
                   (278-offset,8),
                   (279-offset,9),
                   (285-offset,5),
                   (285-offset,-10),
                   (295-offset,-20),
                   (-180,-20),
                   (-180,66),
                  ]

north_pacific_west = get_mask(poly_verts_west, lsmask)
north_pacific_east = get_mask(poly_verts_east, lsmask)

### North Atlantic 

In [ ]:
offset = 360
poly_verts = [(285-offset,66), 
              (290-offset,50),
              (280-offset,40),
              (260-offset,30),
              (260-offset,20),
              (270-offset,15),
              (276-offset,12),
              (278-offset,8),
              (279-offset,9),
              (285-offset,6),
              (355-offset,6),
              (354-offset,35),
              (362-offset,45),
              (376-offset,60),
              (375-offset,66),
              (285-offset,66),
             ]

north_atlantic = get_mask(poly_verts, lsmask)

### South Pacific

In [ ]:
offset = 360
poly_verts = [(145,-20), 
              (295-offset,-20),
              (293-offset,-30),
              (290-offset,-40),
              (288-offset,-50),
              (294-offset,-71),
              (135,-71),
              (145,-20),
             ]

poly_verts_east = [(145,-20),
                   (135,-71),
                   (180, -71),
                   (180, -20),
                   (145,-20),
                  ]

poly_verts_west = [(-180, -20),
                   (295-offset,-20),
                  (293-offset,-30),
                  (290-offset,-40),
                  (288-offset,-50),
                  (294-offset,-71),
                  (-180,-71),]

south_pacific = get_mask(poly_verts, lsmask)
south_pacific_east = get_mask(poly_verts_east, lsmask)
south_pacific_west = get_mask(poly_verts_west, lsmask)

### Indian Ocean

In [ ]:
poly_verts = [(135,-71),
              (145,-20),
              (142,-5),
              (130,0),
              (125,5),
              (125,10),
              (120,20),
              (115,30),
              (60,30),
              (56,20),
              (45,13),
              (35,0),
              (20,-30),
              (20,-71),
              (135,-71)
             ]

indian_ocean = get_mask(poly_verts, lsmask)

### South Atlantic

In [ ]:
offset = 360
poly_verts = [(380-offset,-71),
              (380-offset,30),
              (380-offset,-71),
              (380-offset,-10),
              (375-offset,6),
              (305-offset,6),
              (295-offset,-30),
              (288-offset,-50),
              (294-offset,-71),
              
              
             ]

south_atlantic = get_mask(poly_verts, lsmask)

In [ ]:
fig = plt.figure(figsize=(10,5)) 
ax = plt.subplot(111, projection=ccrs.Robinson(central_longitude=-160))
ax.set_global()
ax.add_feature(cfeature.LAND, facecolor='k', zorder=1) 
lsmask.plot.contourf(levels=3, colors='none', hatches=['////',None], transform=ccrs.PlateCarree(central_longitude=-180), 
                     add_colorbar=False, add_labels=False)
# plt.plot(xlon, ylat, c='orange', lw=2,transform=ccrs.PlateCarree())
north_pacific_east.plot.contourf(transform=ccrs.PlateCarree(), colors='orange', add_colorbar=False)
north_pacific_west.plot.contourf(transform=ccrs.PlateCarree(), colors='orange', add_colorbar=False)
north_atlantic.plot.contourf(transform=ccrs.PlateCarree(), colors='mediumpurple', add_colorbar=False)
south_pacific_east.plot.contourf(transform=ccrs.PlateCarree(), colors='seagreen', add_colorbar=False)
south_pacific_west.plot.contourf(transform=ccrs.PlateCarree(), colors='seagreen', add_colorbar=False)
indian_ocean.plot.contourf(transform=ccrs.PlateCarree(), colors='powderblue', add_colorbar=False)
south_atlantic.plot.contourf(transform=ccrs.PlateCarree(), colors='gold', add_colorbar=False)

# plt.savefig('../figures/basin_map.png', transparent=True)

Load OISST detrended anomalies

In [ ]:
ds_oisst = xr.open_dataset('~/marine-heatwaves/data/preprocessed_oisst.nc')
ssta_notrend = ds_oisst.ssta_notrend


Load Ocetrac labels

In [ ]:
ds_ocetrac = xr.open_dataset('~/marine-heatwaves/data/ocetrac_labels.nc')
